<h1>Sentiment analysis</h1>
<ol>
<h3><li>Définition du Sujet et Objectif</h3></li>
<b>Sentiment analysis ?</b><br>
<p>Aussi connue sous le nom de <strong><i>« Opinion Mining »</i></strong>, <strong>l’analyse des sentiments</strong> consiste à identifier les informations subjectives d’un texte pour <strong>extraire l’opinion de l’auteur</strong>. <br>
De manière générale, <strong>l’analyse des sentiments</strong> permet de mesurer le niveau de satisfaction des clients vis-à-vis des produits ou services fournis par une entreprise ou un organisme. Elle peut même s’avérer <strong>bien plus efficace que des méthodes classiques</strong> comme les sondages puisque <strong>de nos jours, une partie croissante des consommateurs partage fréquemment leurs opinions sur les réseaux sociaux</strong>. </p>

<p><i><strong>Objectif :</strong></i> Le but de notre projet est donc de mettre en place un modèle permettant de classifier les différents avis des clients d’un hôtel en deux classes qui sont : Avis positifs et avis négatifs.<br>
Cette classification permettra au personnel de l’hôtel de pouvoir identifier les principales plaintes (Texte négatifs) afin d’améliorer leurs services et réduire le niveau d’insatisfaction des clients.<br></p>

<h3><li>Constitution de la dataset</h3></li>
<p>Avant d’entamer le travail proprement dit, nous allons d’abord décrire brièvement notre dataset.<br>
Nous avons utilisé une dataset téléchargeable sur Kaggle via le lien suivant : 
<a href="https://www.kaggle.com/andrewmvd/trip-advisor-hotel-reviews">https://www.kaggle.com/andrewmvd/trip-advisor-hotel-reviews</a><br></p>
Elle contient 15 000 lignes et deux colonnes : <br>
<ul>
    <li>Une colonne Review : qui n’est autre qu’un paragraphe contenant le commentaire de l’utilisateur</li>
    <li>Une colonne Rating : qui précise s’il s’agit d’un avis positif ou pas. 1 pour les avis positifs et 0 pour les avis négatifs</li>
</ul><br>
<p>En affichant le résumé de la dataset, on s’aperçoit qu’elle contient plus d’avis positifs (17000) que d’avis négatifs (3000). Cette différence pourra sans doute influencer le modèle lors de l’entrainement. Ce dernier risque donc d’overfitter sur les avis positifs.</p><br>
Pour remédier à cela, nous avons pensé à deux choses :
<ul>
    <li>Soit réduire le nombre d’avis positifs </li>
    <li>Soit faire du webScraping afin d’augmenter le nombre d’avis négatifs.</li>
</ul>

La première solution nous permettra d’obtenir au finish une dataset d’environ 6000 lignes, ce qui n’est pas du tout approprié pour un problème de NLP.<br>
Nous avons donc opté pour la 2ème solution, celle du webScraping.<br>
Nous avons scrapper plusieurs sites différents. Les détails du webscraping se trouve dans le notebook <a href="scraping_data.py"> webscraping.ipynb</a>.

Ensuite nous avons concaténé les deux dataframes pour n’en faire qu’une seule.<br>
//code 
Une fois que cela est fait, Notre dataset est constitué et on peut enchainer avec les étapes suivantes.<br>
<h3><li>Plan de travail</h3></li>
Comme on peut s’en douter, Il s’agit ici d’un problème de Natural Language Processing (NLP) ou Traitement automatique du langage naturel.<br>
Et comme pour tout problème de NLP, Notre travail se compose principalement de deux parties :

<ul>
    <li>La partie <b>« linguistique »</b>, qui consiste à prétraiter et transformer les informations en entrée en un jeu de données exploitable.</li>
    <li>La partie <b>« apprentissage automatique »</b>, qui porte sur l’application de modèles <b><i>de Machine Learning</i></b> à ce jeu de données.</li><br>
</ul>
Dans la suite du rapport, Nous allons aborder ces deux aspects, en décrivant brièvement les <b>principales méthodes utilisées </b>et en précisant les principaux défis auxquels nous avons fait face.
</ol>

<ol type="A">
<h3><li>Partie linguistique : Du texte à la donnée</li></h3>
<p>Parmi les principales étapes de cette partie, on retrouve :</p>
<ol type="a">
<li>Nettoyage : cette phase consiste à réaliser des tâches telles que la <b><i>suppression d’urls, d’emoji, suppression des chiffres, ponctuation, symboles et stopwords, passage en minuscule</i><b>.</li>
</ol>

In [1]:
import re
import string
from nltk.corpus import stopwords

def remove_URL (text):
    url = re.compile(r"https?://\s+ www\.\s+")
    return url.sub(r"", text)


def remove_html(text):
    html = re.compile(r"<.*?>")
    return html.sub(r"", text)
         

def remove_emoji(string):
    emoji_pattern = re.compile(
        "["
        u"\U0001F600-\U0001F64F" # emoticons
        u"\U0001F300-\U0001F5FF" # symbols & pictographs
        u"\U0001F680-\U0001F6FF" # transport & map symbols
        u"\U0001F1E0-\U0001F1FF" # flags (i0s)
        u"\U00002702-\U000027B0"
        u"\U000024C2-\U0001F251"
        "]+",
        flags=re.UNICODE,
    )
    return emoji_pattern.sub(r"", string)


def remove_punct (text):
    table = str.maketrans ("","",string.punctuation)
    return text.translate(table)

stop = set (stopwords.words ( "english"))
def remove_stopwords (text):
    text = [word.lower () for word in text.split() if word.lower() not in stop]
    return " ".join (text)

ModuleNotFoundError: No module named 'nltk'

<p>On regroupe toutes ses fonctions en une seule appelée ‘nettoyer dataframe’</p>


In [ ]:
def nettoyerDataframe(df):
    df["Review"] = df.Review.map(lambda x: remove_URL(x))
    df["Review"] = df.Review.map(lambda x: remove_html(x))
    df["Review"] = df.Review.map(lambda x: remove_emoji(x))
    df["Review"] = df.Review.map(lambda x: remove_punct(x)) 
    df["Review"] = df.Review.map(remove_stopwords)   
    return df

<ol type="a" start="2">
<li>Normalisation des données :</li>
<b><i>Tokenisation</i></b>, ou découpage du texte en plusieurs pièces appelés tokens.<br>
Pour cela, nous utiliserons la méthode des N-grammes.<br>
Les N-grammes sont simplement toutes les combinaisons de mots ou de lettres adjacents de longueur n que nous pouvons trouver dans notre texte source. Les ngrammes avec n=1 sont appelés <i>unigrammes</i>. De même, <i>les bigrammes</i>(n=2), les trigrammes (n=3) et ainsi de suite peuvent également être utilisés.
<br><br>
Les unigrammes ne contiennent généralement pas beaucoup d'informations par rapport aux bigrammes et aux trigrammes. Le principe de base derrière les n-grammes est qu'ils capturent la lettre ou le mot susceptible de suivre le mot donné. Plus le n-gramme est long (n élevé), plus vous devez travailler avec du contexte.
<br><br>
Dans le cas de notre projet, Nous utiliserons les <b><i>bigrammes</i></b> puisque :
<ul>
<li>Avec les unigrames, chaque mot est isolé de son contexte. Ce qui rendra sans doute moins efficace notre modéle.</li>
<li>L’utilisation des N-grammes avec N>2 conduit à une Memory Error (Mémoire RAM insuffisante).</li>
</ul>
<br><br>
<li> Ensuite, Afin de pouvoir appliquer les méthodes de <i>Machine Learning</i> aux problèmes relatifs au langage naturel, il est indispensable de <i>transformer les données textuelles en données numériques</i>.<br>Pour se faire, Il existe plusieurs approches. L’une d’entre elles que nous utiliserons dans ce projet est celle du TF-IDF (<i>Term Frequency-Inverse Document Frequency</i>). Cette méthode consiste <b>à compter le nombre d’occurrences des tokens</b> présents dans le corpus pour chaque texte, que l’on divise ensuite par le nombre d’occurrences total de ces même tokens dans tout le corpus.</li>
</ol>

In [ ]:

def tfidf(df,ngrams):
    vect = TfidfVectorizer(ngram_range=ngrams).fit(df["Review"])
    X = vect.transform(df["Review"])
    return X


<p>A présent nous allons appliquer toutes ces méthodes à la colonne Review de notre dataset afin de la nettoyer et la convertir en TF-IDF.</p>


In [ ]:
df = pd.read_csv('tripadvisor_hotel_reviews.csv')

df['Rating'] = df['Rating'].map({1:0, 2:0, 3:1, 4:1, 5:1})

df = nettoyerDataframe(df)

X = tfidf(df,(1,1))

y=df['Rating']

<ol type="A" start="2">
<h3><li>La phase d'apprentissage: Des données au modèle</li></h3>
<p>De manière globale, on peut distinguer <b>3 principales approches NLP</b> : les <b>méthodes basées sur des règles</b>, modèles classiques de Machine Learning et modèles de Deep Learning.</p>
<br>
<ol type="a">
<li>Modèles utilisés</li>
<p>Nous utiliserons uniquement les modèles classiques de Machine Learning.<br>Elles mettent généralement en œuvre un modèle <b>statistique d’apprentissage automatique</b> tels que ceux de <b>Naive Bayes</b>, de <b>Régression Logistique</b>, <b>SVM</b>.
<br><br>
Nous utiliserons les trois modèles précités et nous ferons une comparaison des résultats.<br><br>
Mais avant cela, Il est nécessaire de subdiviser son jeu de données en jeu d’entrainement et de test.</p>
</ol>


</ol>

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

<p>Ensuite on instancie le modèle, on l’entraîne, on prédit les résultats sur le jeu de test, puis on évalue la performance à l’aide de quelques métriques.</p>


<ol type="a" start="2">
<li>Métriques d’évaluation </li>
<br>
Après, Donc, afin d'évaluer les modèles de classification, nous discuterons de ces métriques en détail :
<ul>
<li>Précision</li>
<li>Précision et rappel</li>
<li>Score F1</li>
</ul>
</ol>

<h5>Regression logistique</h5>


In [ ]:
logisticRegression = LogisticRegression(multi_class='multinomial')
logisticRegression.fit(X_train, y_train)
y_pred = logisticRegression.predict(X_test)

print("\n***** Métriques d'évaluation : Regression logistique ******")
f1score = f1_score(y_test, y_pred, average='micro')
accuracy = accuracy_score(y_test, y_pred)
print (f"Score: {f1score * 100} %") 
print(f"Accuracy : {accuracy * 100} %")

<h5>Naive Bayes</h5>

In [ ]:
from sklearn.naive_bayes import GaussianNB

gaussianNB = GaussianNB()
gaussianNB.fit(X_train.toarray(),y_train)

y_pred = logisticRegression.predict(X_test)

print("\n***** Métriques d'évaluation : Naives Bayes ******")
f1score = f1_score(y_test, y_pred, average='micro')
accuracy = accuracy_score(y_test, y_pred)
print (f"Score: {f1score * 100} %") 
print(f"Accuracy : {accuracy * 100} %")


<h5>SVM</h5>

In [ ]:
from sklearn import svm

svmModel = svm.SVC(kernel='linear') # Linear Kernel
svmModel.fit(X_train, y_train)

y_pred = svmModel.predict(X_test)
print("\n***** Métriques d'évaluation : SVM ******")
f1score = f1_score(y_test, y_pred, average='micro')
accuracy = accuracy_score(y_test, y_pred)
print (f"Score: {f1score * 100} %") 
print(f"Accuracy : {accuracy * 100} %")


<h4>Validation croisée (uniquement pour la regression logiqtique) :</h4>
<br>
Supposons que nous n'avons pas beaucoup de données, et cela n'a pas de sens de diviser nos données en train, validation et test. Sklearn a un objet cross_val_score qui nous permet de voir dans quelle mesure notre modèle se généralise.<br>
Nous allons donc l’appliquer sur notre dataset sans la diviser en jeu de données et de tests.

In [ ]:
scores = cross_val_score(logisticRegression, X, df["Rating"], cv=10)

print('Cross-Validation Accuracy Scores', scores)
scores = pd.Series(scores)
print(scores.min())
print(scores.mean())
print(scores.max())
